In [1]:
#To allocate memorey for gpu(here we are using 50%)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [3]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential

Using TensorFlow backend.


In [4]:
import numpy as np
from glob import glob

In [5]:
#resize all images
IMAGE_SIZE = [224,224]
train_path = 'train'
test_path = 'test'

In [6]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top=False)

In [7]:
#dont train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [8]:
train_folders = glob('train/*')
train_folders

['train\\female', 'train\\male']

In [9]:
test_folders = glob('test/*')
test_folders

['test\\female', 'test\\male']

In [10]:
#flattern layer
x = Flatten()(vgg16.output)

In [11]:
prediction = Dense(2,activation='softmax')(x)
#create model
model = Model(inputs=vgg16.input,outputs=prediction)

In [12]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip = True)

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
training_set = train_datagen.flow_from_directory('train',
                                                 target_size=(224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1849 images belonging to 2 classes.


In [17]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 458 images belonging to 2 classes.


In [18]:
#fit the model
r = model.fit_generator(training_set,
                        validation_data = test_set,
                        epochs=20,
                        steps_per_epoch = len(training_set),
                        validation_steps = len(test_set))

Epoch 1/20
58/58 [==============================] - 99s 2s/step - loss: 0.5072 - accuracy: 0.7918 - val_loss: 0.1588 - val_accuracy: 0.8865
Epoch 2/20
58/58 [==============================] - 22s 376ms/step - loss: 0.1352 - accuracy: 0.9508 - val_loss: 0.0594 - val_accuracy: 0.9301
Epoch 3/20
58/58 [==============================] - 22s 376ms/step - loss: 0.1075 - accuracy: 0.9621 - val_loss: 0.4502 - val_accuracy: 0.9301
Epoch 4/20
58/58 [==============================] - 34s 578ms/step - loss: 0.1001 - accuracy: 0.9621 - val_loss: 0.1274 - val_accuracy: 0.9367
Epoch 5/20
58/58 [==============================] - 26s 441ms/step - loss: 0.0769 - accuracy: 0.9735 - val_loss: 0.5972 - val_accuracy: 0.9345
Epoch 6/20
58/58 [==============================] - 26s 455ms/step - loss: 0.0682 - accuracy: 0.9789 - val_loss: 0.3448 - val_accuracy: 0.9454
Epoch 7/20
58/58 [==============================] - 22s 379ms/step - loss: 0.0623 - accuracy: 0.9773 - val_loss: 0.0149 - val_accuracy: 0.9410
Ep

In [19]:
from keras.models import load_model
model.save('Gender_Detection_vgg16.h5')